Data source - https://www.kaggle.com/beradabhishek/europe-economy

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd

In [ ]:
alt.data_transformers.enable('default', max_rows=None) 

# Read data

In [ ]:
economy = pd.read_csv('data/economy.csv')
geo_data = gpd.read_file('data/europe.geojson')

In [ ]:
economy = economy[economy.Country != 'European Union']
economy = economy[economy.Country != 'Eurozone']
df = economy

In [ ]:
font = 'Ubuntu Mono'
red = '#d53e4f'
blue = '#3288bd'
gray = '#5D646F'

# Graph 1: How economy level changed in Europe from 2012 to 2017

In [ ]:
economy.head()

In [ ]:
chart1 = alt.Chart(economy).mark_line().encode(
    x = alt.X('2012'),
    x2 = alt.X2('2017'),
    y = alt.Y('Country', sort=alt.Sort(field='2017', op='max', order='descending'), title=None)
)

chart2 = alt.Chart(economy).mark_circle().encode(
    x = alt.X('2012', title='economy level'),
    y = alt.Y('Country', sort=alt.Sort(field='2017', op='max', order='descending'))
   # color  = alt.Color('year:N')
)

chart3 = alt.Chart(economy).mark_circle().encode(
    x = alt.X('2017'),
    y = alt.Y('Country', sort=alt.Sort(field='2017', op='max', order='descending')),
    color=alt.value('red')
)

chart = chart1 + chart2 + chart3
chart.properties(background = '#EFECFA', padding = 25, title = alt.TitleParams(text = 'Changes in EU countries economy from 2012 to 2017', 
            subtitle = ['Blue points are for 2012 and red - 2017', ' '],
            font = font, fontSize=20, color='#635F5D', subtitleFont = font, anchor = 'start',
            subtitleFontSize = 12, subtitleColor = '#8E8883', subtitleFontStyle = 'italic')
).configure_axis(domain = False, ticks = False, gridColor = 'white', gridWidth = 1,
            labelFont = font, labelFontSize = 12, labelColor = '#635F5D', labelPadding = 5, labelFontWeight = 'bold',
            titleFont = font, titleFontSize = 12, titleFontWeight = 'normal', titleColor = '#635F5D', titlePadding = 22
).configure_view(strokeOpacity = 0, width = 650,)

# Graph 2: Comparing growth level of European countries

In [ ]:
missing = pd.DataFrame([['Ukraine', -1.5], ['Switzerland', -1.5], ['Iceland', -1.5], ['Republic of Moldova', -1.5], ['Belarus', -1.5], ['Russia', -1.5], ['Turkey', -1.5], ['Georgia', -1.5]], columns=['Country', 'Yearly Growth(2012-2017)'])
economy = economy.append(missing, ignore_index=True, sort=True)
economy.Country.replace(['Czechia'], ['Czech Republic'], inplace=True)

In [ ]:
alt.Chart(geo_data, width=400, height=300).mark_geoshape().encode(
    color=alt.condition('datum["Yearly Growth(2012-2017)"] >= -1', alt.Color('Yearly Growth(2012-2017):Q'), alt.value('lightgray')),
    tooltip = alt.Tooltip("NAME:N")
).project(
    'naturalEarth1'
).transform_lookup(
    lookup='NAME',
    from_=alt.LookupData(economy, 'Country', ['Yearly Growth(2012-2017)'])
).properties(background = '#EFECFA', padding = 25, title = alt.TitleParams(text = 'Ireland has the biggest yearly growth level', 
            subtitle = ['2012-2017', ' '],
            font = font, fontSize=20, color='#635F5D', subtitleFont = font, anchor = 'start',
            subtitleFontSize = 12, subtitleColor = '#8E8883', subtitleFontStyle = 'italic')
).configure_axis(domain = False, ticks = False, gridColor = 'white', gridWidth = 0,
            labelFont = font, labelFontSize = 12, labelColor = '#635F5D', labelPadding = 5, labelFontWeight = 'bold',
            titleFont = font, titleFontSize = 12, titleFontWeight = 'normal', titleColor = '#635F5D', titlePadding = 22
).configure_view(strokeOpacity = 0, width = 650,
).configure_legend(
            labelColor = '#635F5D', titleColor = '#635F5D', fillColor='#E5E2E0', padding=10,
            strokeColor='#C0C0BB', title=None, orient='right'
)

# Chart 3: Economy changes for every country

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'LastAvailableQuarter', 'Yearly Growth(2012-2017)', '2018'])
df1 = pd.melt(df, id_vars=['Country'], value_vars=['2012', '2013', '2014', '2015', '2016', '2017'], value_name='value')[['Country', 'value']]

In [ ]:
lst = []

for year in ['2012', '2013', '2014', '2015', '2016', '2017']:
  lst += [int(year) for i in range (28)]

df1['year'] = lst

In [ ]:
countries = list(df.Country.unique())
country_dropdown = alt.binding_select(options=countries)
country_select = alt.selection_single(name="Choose", fields=['Country'], bind=country_dropdown, init={'Country': 'Italy'})

alt.Chart(df1).mark_bar(size=7).encode(
    x = alt.X('year:O', title=None, axis=alt.Axis(labelAngle=0)),
    y = alt.Y('value', title=None),
    color=alt.condition('datum.value >= 0', alt.value(blue), alt.value(red)),
).add_selection(
    country_select
).transform_filter(
    country_select
).properties(background = '#EFECFA', padding = 25, title = alt.TitleParams(text = 'Check out growth level of European countries', 
            subtitle = ['2012-2017', ' '],
            font = font, fontSize=20, color='#635F5D', subtitleFont = font, anchor = 'start',
            subtitleFontSize = 12, subtitleColor = '#8E8883', subtitleFontStyle = 'italic')
).configure_axis(domain = False, ticks = False, gridColor = 'white', gridWidth = 1,
            labelFont = font, labelFontSize = 12, labelColor = '#635F5D', labelPadding = 5, labelFontWeight = 'bold',
            titleFont = font, titleFontSize = 12, titleFontWeight = 'normal', titleColor = '#635F5D', titlePadding = 22
).configure_view(strokeOpacity = 0, width = 650,
).configure_legend(
            labelColor = '#635F5D', titleColor = '#635F5D', fillColor='#E5E2E0', padding=10,
            strokeColor='#C0C0BB', title=None, orient='right'
)